# Mars Game Experiment Analysis

Comprehensive analysis of Mars resource management game experiments.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add src to path
sys.path.append(str(Path.cwd().parent.parent / "src"))

from data_processing import process_simulation_results, load_config

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Paths
RESULTS_DIR = Path.cwd().parent.parent / "results"
PLOTS_DIR = RESULTS_DIR / "plots"
PLOTS_DIR.mkdir(exist_ok=True)

## 1. Load and Explore Data

In [ ]:
# Load experiment results
result_files = list(RESULTS_DIR.glob("*.csv"))
print(f"Found {len(result_files)} result files:")
for file in result_files[:5]:  # Show first 5
    print(f"  - {file.name}")

# Load the most recent or combined results
if result_files:
    # Try to find combined results first
    combined_files = [f for f in result_files if 'combined' in f.name.lower()]
    if combined_files:
        df = pd.read_csv(combined_files[-1])  # Most recent combined
        print(f"\nLoaded combined results: {combined_files[-1].name}")
    else:
        df = pd.read_csv(result_files[-1])  # Most recent individual
        print(f"\nLoaded individual results: {result_files[-1].name}")
    
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
else:
    print("No result files found. Please run some experiments first.")
    df = pd.DataFrame()  # Empty dataframe for demo

In [ ]:
# Basic data exploration
if not df.empty:
    print("Dataset Overview:")
    print(df.describe())
    
    print("\nMissing Values:")
    print(df.isnull().sum())
    
    print("\nUnique Values:")
    for col in ['model', 'risk_level']:
        if col in df.columns:
            print(f"{col}: {df[col].nunique()} unique values")
            print(f"  Values: {sorted(df[col].dropna().unique())}")

## 2. Overall Performance Analysis

In [ ]:
if not df.empty:
    # Overall statistics
    total_runs = len(df)
    successful_runs = df['failure_round'].isna().sum()
    success_rate = successful_runs / total_runs
    
    print(f"Overall Performance:")
    print(f"  Total Runs: {total_runs}")
    print(f"  Successful Runs: {successful_runs} ({success_rate:.1%})")
    print(f"  Average Score: {df['final_score'].mean():.2f} ± {df['final_score'].std():.2f}")
    print(f"  Median Score: {df['final_score'].median():.2f}")
    print(f"  Average Survival: {df['survival_rounds'].mean():.1f} rounds")
    
    # Score distribution
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Score histogram
    axes[0,0].hist(df['final_score'], bins=30, alpha=0.7, edgecolor='black')
    axes[0,0].axvline(df['final_score'].mean(), color='red', linestyle='--', label='Mean')
    axes[0,0].axvline(df['final_score'].median(), color='orange', linestyle='--', label='Median')
    axes[0,0].set_xlabel('Final Score')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].set_title('Distribution of Final Scores')
    axes[0,0].legend()
    
    # Survival rounds
    survival_counts = df['survival_rounds'].value_counts().sort_index()
    axes[0,1].bar(survival_counts.index, survival_counts.values, alpha=0.7)
    axes[0,1].set_xlabel('Survival Rounds')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].set_title('Distribution of Survival Rounds')
    
    # Investment vs Score
    axes[1,0].scatter(df['average_investment'], df['final_score'], alpha=0.6)
    axes[1,0].set_xlabel('Average Investment per Round')
    axes[1,0].set_ylabel('Final Score')
    axes[1,0].set_title('Investment Strategy vs Performance')
    
    # Health management
    if 'min_health' in df.columns:
        axes[1,1].scatter(df['min_health'], df['final_score'], alpha=0.6)
        axes[1,1].set_xlabel('Minimum Health Reached')
        axes[1,1].set_ylabel('Final Score')
        axes[1,1].set_title('Health Management vs Performance')
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'overall_performance.png', dpi=300, bbox_inches='tight')
    plt.show()

## 3. Model Comparison

In [ ]:
if not df.empty and 'model' in df.columns and df['model'].nunique() > 1:
    # Model comparison
    model_stats = df.groupby('model').agg({
        'final_score': ['count', 'mean', 'std', 'median'],
        'failure_round': lambda x: x.isna().mean(),  # Success rate
        'survival_rounds': 'mean',
        'average_investment': 'mean'
    }).round(2)
    
    model_stats.columns = ['Runs', 'Mean_Score', 'Std_Score', 'Median_Score', 'Success_Rate', 'Avg_Survival', 'Avg_Investment']
    print("Model Comparison:")
    print(model_stats)
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Score comparison
    df.boxplot(column='final_score', by='model', ax=axes[0,0])
    axes[0,0].set_title('Score Distribution by Model')
    axes[0,0].set_xlabel('Model')
    axes[0,0].set_ylabel('Final Score')
    
    # Success rate
    success_rates = df.groupby('model')['failure_round'].apply(lambda x: x.isna().mean())
    axes[0,1].bar(success_rates.index, success_rates.values, alpha=0.7)
    axes[0,1].set_title('Success Rate by Model')
    axes[0,1].set_xlabel('Model')
    axes[0,1].set_ylabel('Success Rate')
    axes[0,1].set_ylim(0, 1)
    
    # Investment strategy
    df.boxplot(column='average_investment', by='model', ax=axes[1,0])
    axes[1,0].set_title('Investment Strategy by Model')
    axes[1,0].set_xlabel('Model')
    axes[1,0].set_ylabel('Average Investment')
    
    # Survival rounds
    df.boxplot(column='survival_rounds', by='model', ax=axes[1,1])
    axes[1,1].set_title('Survival Rounds by Model')
    axes[1,1].set_xlabel('Model')
    axes[1,1].set_ylabel('Survival Rounds')
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print("No model comparison data available (single model or no model column)")

## 4. Risk Profile Analysis

In [ ]:
if not df.empty and 'risk_level' in df.columns and df['risk_level'].notna().any():
    # Risk level analysis
    risk_df = df[df['risk_level'].notna()].copy()
    
    risk_stats = risk_df.groupby('risk_level').agg({
        'final_score': ['count', 'mean', 'std'],
        'failure_round': lambda x: x.isna().mean(),
        'average_investment': 'mean',
        'survival_rounds': 'mean'
    }).round(2)
    
    risk_stats.columns = ['Runs', 'Mean_Score', 'Std_Score', 'Success_Rate', 'Avg_Investment', 'Avg_Survival']
    print("Risk Level Analysis:")
    print(risk_stats)
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Score vs risk level
    risk_means = risk_df.groupby('risk_level')['final_score'].mean()
    risk_stds = risk_df.groupby('risk_level')['final_score'].std()
    
    axes[0,0].errorbar(risk_means.index, risk_means.values, yerr=risk_stds.values, 
                      marker='o', capsize=5, capthick=2)
    axes[0,0].set_xlabel('Risk Level')
    axes[0,0].set_ylabel('Average Final Score')
    axes[0,0].set_title('Performance vs Risk Tolerance')
    axes[0,0].grid(True, alpha=0.3)
    
    # Success rate vs risk level
    success_by_risk = risk_df.groupby('risk_level')['failure_round'].apply(lambda x: x.isna().mean())
    axes[0,1].plot(success_by_risk.index, success_by_risk.values, marker='o', linewidth=2)
    axes[0,1].set_xlabel('Risk Level')
    axes[0,1].set_ylabel('Success Rate')
    axes[0,1].set_title('Success Rate vs Risk Tolerance')
    axes[0,1].set_ylim(0, 1)
    axes[0,1].grid(True, alpha=0.3)
    
    # Investment strategy vs risk level
    investment_by_risk = risk_df.groupby('risk_level')['average_investment'].mean()
    axes[1,0].plot(investment_by_risk.index, investment_by_risk.values, marker='o', linewidth=2)
    axes[1,0].set_xlabel('Risk Level')
    axes[1,0].set_ylabel('Average Investment')
    axes[1,0].set_title('Investment Strategy vs Risk Tolerance')
    axes[1,0].grid(True, alpha=0.3)
    
    # Distribution of scores by risk category
    risk_df['risk_category'] = pd.cut(risk_df['risk_level'], 
                                     bins=[0, 3, 6, 10], 
                                     labels=['Risk Averse', 'Moderate', 'Risk Seeking'])
    
    risk_df.boxplot(column='final_score', by='risk_category', ax=axes[1,1])
    axes[1,1].set_title('Score Distribution by Risk Category')
    axes[1,1].set_xlabel('Risk Category')
    axes[1,1].set_ylabel('Final Score')
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'risk_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print("No risk profile data available")

## 5. Strategy Analysis

In [ ]:
if not df.empty:
    # Strategy clustering based on investment patterns
    print("Strategy Analysis:")
    
    # Define strategy categories based on investment behavior
    df['strategy'] = 'Balanced'
    df.loc[df['average_investment'] < 4, 'strategy'] = 'Aggressive (Low Investment)'
    df.loc[df['average_investment'] > 7, 'strategy'] = 'Conservative (High Investment)'
    
    strategy_stats = df.groupby('strategy').agg({
        'final_score': ['count', 'mean', 'std'],
        'failure_round': lambda x: x.isna().mean(),
        'survival_rounds': 'mean',
        'min_health': 'mean' if 'min_health' in df.columns else lambda x: np.nan
    }).round(2)
    
    print(strategy_stats)
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Strategy distribution
    strategy_counts = df['strategy'].value_counts()
    axes[0,0].pie(strategy_counts.values, labels=strategy_counts.index, autopct='%1.1f%%')
    axes[0,0].set_title('Distribution of Strategies')
    
    # Performance by strategy
    df.boxplot(column='final_score', by='strategy', ax=axes[0,1])
    axes[0,1].set_title('Performance by Strategy')
    axes[0,1].set_xlabel('Strategy')
    axes[0,1].set_ylabel('Final Score')
    
    # Investment vs Health Management
    if 'min_health' in df.columns:
        scatter = axes[1,0].scatter(df['average_investment'], df['min_health'], 
                                   c=df['final_score'], cmap='viridis', alpha=0.6)
        axes[1,0].set_xlabel('Average Investment')
        axes[1,0].set_ylabel('Minimum Health')
        axes[1,0].set_title('Investment vs Health Management\n(Color = Final Score)')
        plt.colorbar(scatter, ax=axes[1,0])
    
    # Success rate by strategy
    success_by_strategy = df.groupby('strategy')['failure_round'].apply(lambda x: x.isna().mean())
    axes[1,1].bar(success_by_strategy.index, success_by_strategy.values, alpha=0.7)
    axes[1,1].set_title('Success Rate by Strategy')
    axes[1,1].set_xlabel('Strategy')
    axes[1,1].set_ylabel('Success Rate')
    axes[1,1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'strategy_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

## 6. Statistical Analysis

In [ ]:
if not df.empty:
    from scipy import stats
    
    print("Statistical Analysis:")
    print("=" * 50)
    
    # Correlation analysis
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    correlation_matrix = df[numeric_cols].corr()
    
    # Focus on correlations with final_score
    if 'final_score' in correlation_matrix.columns:
        score_correlations = correlation_matrix['final_score'].sort_values(key=abs, ascending=False)
        print("\nCorrelations with Final Score:")
        for var, corr in score_correlations.items():
            if var != 'final_score':
                print(f"  {var}: {corr:.3f}")
    
    # Statistical tests
    if 'model' in df.columns and df['model'].nunique() > 1:
        print("\nModel Comparison (ANOVA):")
        models = df['model'].unique()
        model_scores = [df[df['model'] == model]['final_score'].values for model in models]
        f_stat, p_value = stats.f_oneway(*model_scores)
        print(f"  F-statistic: {f_stat:.3f}")
        print(f"  p-value: {p_value:.3f}")
        print(f"  Significant difference: {'Yes' if p_value < 0.05 else 'No'}")
    
    if 'risk_level' in df.columns and df['risk_level'].notna().any():
        print("\nRisk Level vs Performance (Correlation):")
        risk_df = df[df['risk_level'].notna()]
        corr_coef, p_value = stats.pearsonr(risk_df['risk_level'], risk_df['final_score'])
        print(f"  Correlation coefficient: {corr_coef:.3f}")
        print(f"  p-value: {p_value:.3f}")
        print(f"  Significant correlation: {'Yes' if p_value < 0.05 else 'No'}")
    
    # Heatmap of correlations
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": .8})
    plt.title('Correlation Matrix of Numeric Variables')
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

## 7. Summary and Insights

In [ ]:
if not df.empty:
    print("KEY INSIGHTS:")
    print("=" * 50)
    
    # Overall performance insights
    success_rate = df['failure_round'].isna().mean()
    avg_score = df['final_score'].mean()
    
    print(f"\n1. OVERALL PERFORMANCE:")
    print(f"   • Success Rate: {success_rate:.1%}")
    print(f"   • Average Score: {avg_score:.1f} points")
    print(f"   • Most common failure round: {df['failure_round'].mode().iloc[0] if not df['failure_round'].mode().empty else 'N/A'}")
    
    # Strategy insights
    if 'strategy' in df.columns:
        best_strategy = df.groupby('strategy')['final_score'].mean().idxmax()
        print(f"\n2. STRATEGY INSIGHTS:")
        print(f"   • Best performing strategy: {best_strategy}")
        print(f"   • Investment sweet spot: {df['average_investment'].quantile(0.75):.1f} - {df['average_investment'].quantile(0.9):.1f} units")
    
    # Model insights
    if 'model' in df.columns and df['model'].nunique() > 1:
        best_model = df.groupby('model')['final_score'].mean().idxmax()
        print(f"\n3. MODEL COMPARISON:")
        print(f"   • Best performing model: {best_model}")
        model_success = df.groupby('model')['failure_round'].apply(lambda x: x.isna().mean())
        most_reliable = model_success.idxmax()
        print(f"   • Most reliable model: {most_reliable} ({model_success[most_reliable]:.1%} success)")
    
    # Risk insights
    if 'risk_level' in df.columns and df['risk_level'].notna().any():
        risk_df = df[df['risk_level'].notna()]
        optimal_risk = risk_df.groupby('risk_level')['final_score'].mean().idxmax()
        print(f"\n4. RISK TOLERANCE INSIGHTS:")
        print(f"   • Optimal risk level: {optimal_risk}")
        
        # Risk vs performance correlation
        corr_coef = risk_df['risk_level'].corr(risk_df['final_score'])
        if abs(corr_coef) > 0.1:
            direction = "positively" if corr_coef > 0 else "negatively"
            strength = "strongly" if abs(corr_coef) > 0.3 else "moderately" if abs(corr_coef) > 0.1 else "weakly"
            print(f"   • Risk tolerance is {strength} {direction} correlated with performance (r={corr_coef:.3f})")
    
    print(f"\n5. RECOMMENDATIONS:")
    print(f"   • Target investment range: {df[df['final_score'] > df['final_score'].quantile(0.75)]['average_investment'].mean():.1f} units/round")
    print(f"   • Maintain minimum health above: {df[df['final_score'] > 0]['min_health'].quantile(0.25):.1f} units")
    print(f"   • Focus on surviving past round: {df['failure_round'].quantile(0.5, interpolation='lower') if df['failure_round'].notna().any() else 'N/A'}")
    
    print(f"\nAnalysis complete! Plots saved to: {PLOTS_DIR}")
else:
    print("No data available for analysis. Please run some experiments first.")